In [1]:
import sys
sys.path.append('../')
import utility as util
import copy

config = util.load_specific_yaml('instance_F_1.yaml')
# print(config)
print(config['V'])
print(config['j_amount'])
print(config['k_amount'])
print(config['U_L'])
print(config['U_LT'])

[[467202, 92999], [193013, 204072], [49473, 227435], [257724, 51794], [402197, 13097]]
5
2
[6, 6, 5, 9, 10]
[[2, 4], [2, 4], [2, 4], [3, 2], [3, 1]]


In [2]:
def G_function(x):
  if x > 133:
    return 1
  else:
    return -0.000015 * (x**2) + 0.0095 * x
def E_function(x):
	if x > 100:
		return 40
	else:
	  return -0.004 * (x**2) + 0.8 * x
	
def greedy_best_location(config: dict, candidates: list):
  '''
  輸出：要蓋哪個location,還有要塞哪些車
  '''
  best_util = 0 
  best_to_fill = []
  best_loc = -1
  # Find the best location with most utility.
  for ind, facility in enumerate(candidates):
    if facility == 0:  # haven't built this j
      # start filling with largest V
      car_list = [(value, index) for index, value in enumerate(config['V'][ind])]
      sorted_car_list = sorted(car_list, reverse=True)  # [(488183, 2), (420751, 0), (133684, 1)]
      # The total resource allowed to be used in this location.
      quota_loc = copy.deepcopy(config['U_L'][ind])  # U_L : Max capacity for facility j, Dim : (j)
			# The specific resource allowed to be used in this location.
      quota_loc_k = copy.deepcopy(config['U_LT'][ind])  # U_LT : Max amount of resource k allowed to be allocated to facility j, Dim : (j, k)
			# Max allocation limit for resource k
      quota_k = copy.deepcopy(config['U_T'])  # U_T : Max allocation limit for resource k, Dim : (k)

      cur_to_fill = []
      cur_util = 0
      while quota_loc > 0:  # 一種一種車去填依照效用順序
        # all resources are used up in this location.
        if(len(sorted_car_list) == 0):
          break
        car_to_fill = sorted_car_list.pop(0)
				# The number allowed to fill is the minimum of these 3 limitations.
        num_to_fill = min(quota_loc, quota_loc_k[car_to_fill[1]], quota_k[car_to_fill[1]])
        quota_k[car_to_fill[1]] -= num_to_fill
        quota_loc_k[car_to_fill[1]] -= num_to_fill
        quota_loc -= num_to_fill
        cur_util += num_to_fill * car_to_fill[0]
        # Record the car index and the number of cars to fill.
        cur_to_fill.append((car_to_fill[1], num_to_fill))

      if (cur_util > best_util):
        best_loc = ind
        best_util = cur_util
        best_to_fill = cur_to_fill
        print("New best location found!")
  print(f"Iteration ended! Find the best location: {best_loc}")
  print(f"Best utility: {best_util}")
  return best_to_fill, best_util, best_loc


def update_config_each_iteration_build_j(config: dict, cars_to_fill: list):
  for (car_type, allocate_num) in cars_to_fill:
    print(f"One combination:{car_type}, {allocate_num} (update_config func)")
    config['U_T'][car_type] -= allocate_num
  return config

def calc_current_gain(config: dict, facility_is_built: list):
  total_gain = 0
  for customer_pt_i in range(config["i_amount"]):
    total_attr_i, our_vs_all_percentage = calc_total_attr_i(config, customer_pt_i, facility_is_built)
    total_gain += config['H'][customer_pt_i] * G_function(total_attr_i) * our_vs_all_percentage
  return total_gain

def calc_total_attr_i(config: dict, customer_pt_i: int, facility_is_built: list):
  """
  assume if loc j is built, its E is 40
  ===
  parameters: 
  - customer_pt_i, represent the customer_point i
  - facility_is_built, looks like [0, 1, 0, 0, 1, 1, 1, 1]
  returns:
  - TAi, total attractiveness for this point
  - our_vs_all_percentage, 對於這個顧客i點的貢獻, 我們廠vs全部的比例（P）
  """
  total_attr_i = 0
  # first calculate all our facility contribute to this i
  for j_idx, facility in enumerate(facility_is_built):
    if facility == 1:
      total_attr_i += 40 / (config['D'][customer_pt_i][j_idx] ** 2) 
  only_our_facilities_total_attr = total_attr_i  # 暫時結算 只有我們設施的吸引力 之後拿來算比例
  # add opponent's attr contribute to this i
  for l_idx in range(config['l_amount']):
    total_attr_i += config['A_opponent_bar'][l_idx] / (config['D_comp'][customer_pt_i][l_idx] ** 2)
  return total_attr_i, only_our_facilities_total_attr / total_attr_i


def calc_current_cost(config: dict, facility_is_built: list, compensate_attractiveness: list, cars_usage_record: list):
  total_cost = 0
  build_cost = sum(x * y for x, y in zip(facility_is_built, config['F']))
  attr_cost = sum(x * y for x, y in zip(compensate_attractiveness, config['C']))
  total_cost = total_cost + build_cost + attr_cost
  for (car_type, allocate_num, loc_j) in cars_usage_record:
    total_cost += config['B'][loc_j][car_type] * allocate_num
  return total_cost

In [3]:
# heuristic_greedy
# """
# 進步中 = true
# 最佳W = 0
# 已分配設施 = {...都是0}
# while 此輪還在進步 and 還有工廠能蓋
#     進步中 = false
#     此輪最佳W 暫時= 最佳W（前一輪的）
#     對每個可能的建立工廠點 j:
#         看看哪個車k對於這個點的V最大 一直塞到此點容納不下車or沒車可塞了，
#         剩下用decoration補滿（G最大133.4, E最大100), 此處塞滿E
#         計算總W (包含先前iter已蓋好之工廠點)
#         如果大於此輪最佳，記錄為此輪最佳，更新最佳蓋哪個點 j
#     如果此輪最佳obj > 最佳obj:
#         更新已分配設施用這個最佳設廠點j跟已分配設施
#         更新最佳obj
# """
""" (更新版)
進步中 = true
全局最佳obj = 0
已分配設施 = {...都是0}
while 此輪還在進步 and 還有工廠能蓋
    進步中 = false
    此輪最佳obj 暫時= 最佳obj（前一輪的）
    對每個可能的建立工廠點 j:
        看看哪個車k對於這個點的V最大 一直塞到此點容納不下車or沒車可塞了，
        剩下用decoration補滿（G最大133.4, E最大100), 此處塞滿E
        複製一份config計算總obj (包含先前iter已蓋好之工廠點)
        如果大於此輪最佳，記錄為此輪最佳，更新最佳蓋哪個點 j
    如果此輪最佳obj > 全局最佳obj:
        更新已分配設施用這個最佳設廠點j跟已分配設施
        更新全局最佳obj, 全局config
"""
improve = True
best_obj = 0
candidates = [0 for i in range(config['j_amount'])]
compensate_attractiveness = [0 for i in range(config['j_amount'])]   # addition attr to fulfill e 
cars_usage_record = []  # 紀錄每次分了多少車給哪個j [(1, 5, 18),(0, 4, 18),(2, 3, 18),(2, 5, 2),...] (第k種車, 幾輛, 給哪個j)
while improve: 
	cars_to_fill, get_util_amount, loc_to_build = greedy_best_location(config, candidates)
	if loc_to_build == -1:  # all facilities are built
		break
	print("loc_to_build", loc_to_build)
	candidates[loc_to_build] = 1
	# compensate_attractiveness[loc_to_build] = 100 - get_util_amount   # build bathroom to fulfill e's max(100 util) 先假設沒有建造上限
  compensate_attractiveness[loc_to_build] = min(min(0, 100 - get_util_amount), config['A_EX_bound'])   # build bathroom to fulfill e's max(100 util) 廁所限制加入
	cars_usage_record.extend([(x[0], x[1], loc_to_build) for x in cars_to_fill])  # 紀錄進總共分配掉的車子, 額外紀錄分配給哪個j
	config = update_config_each_iteration_build_j(config, cars_to_fill)  # update config for allocated cars
	cur_obj = calc_current_gain(config, candidates) - calc_current_cost(config, candidates, compensate_attractiveness, cars_usage_record)
	print(f"current obj: {cur_obj}")
	if cur_obj > best_obj:
		best_obj = cur_obj
	else:
		improve = False
"""此版本之最後config不是最佳obj的config(有跟著被改中間才好計算)"""

New best location found!
Iteration ended! Find the best location: 0
Best utility: 1306400
loc_to_build 0
One combination:0, 2 (update_config func)
One combination:1, 4 (update_config func)
current obj: 60.956520009235206
New best location found!
New best location found!
Iteration ended! Find the best location: 4
Best utility: 1219688
loc_to_build 4
One combination:0, 3 (update_config func)
One combination:1, 1 (update_config func)
current obj: 197.601893259627
New best location found!
Iteration ended! Find the best location: 1
Best utility: 1202314
loc_to_build 1
One combination:1, 4 (update_config func)
One combination:0, 2 (update_config func)
current obj: 298.4457340822657
New best location found!
New best location found!
Iteration ended! Find the best location: 3
Best utility: 876760
loc_to_build 3
One combination:0, 3 (update_config func)
One combination:1, 2 (update_config func)
current obj: 340.9259662477723
New best location found!
Iteration ended! Find the best location: 2
Bes

'此版本之最後config不是最佳obj的config(有跟著被改中間才好計算)'

In [6]:
cars_usage_record

[(0, 2, 0),
 (1, 4, 0),
 (0, 3, 4),
 (1, 1, 4),
 (1, 4, 1),
 (0, 2, 1),
 (0, 3, 3),
 (1, 2, 3),
 (1, 1, 2),
 (0, 2, 2)]